In [ ]:
#prophet

In [1]:
!pip install prophet

Defaulting to user installation because normal site-packages is not writeable
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached stanio-0.5.1-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.1 MB 4.3 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/12.1 MB 4.7 MB/s eta 0:00:03
   ---------- ----------------------------- 3.1/12.1 MB 5.4 MB/s eta 0:00:02
   ------------- -------------------------- 4.2/12.1 MB 5.5 MB/s eta 0:00:02
   ------------------ --------------------- 5.5/12.1 MB 5.5 MB/s eta 0:00:02
   ---------------------- ----------------- 6.8/12.1 MB 5.5 MB/s eta 0:00:01
   ------------------------- -------------- 7.9/12.1 MB 5.5 MB/s eta 0:00:01
   ----------------------------- ---------- 8.9/12.1 MB 5.5 MB/s eta 0:00:01
   -------------------------------- ------- 10.0/12.1 MB 5.5 MB/s eta 0:00:01
   ----------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import pandas as pd
import os
from prophet import Prophet
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "prophet_output_models"
output_csv = "prophet_output_csv"
output_graphs = "prophet_output_graphs"
output_metrics_csv = "prophet_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Prepare metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Function to handle multiple date formats
# -----------------------------
def parse_dates(date_series):
    """Try multiple date formats and return parsed dates"""
    for fmt in ("%Y-%m-%d", "%d-%m-%Y", "%d/%m/%Y", "%m/%d/%Y"):
        try:
            return pd.to_datetime(date_series, format=fmt, errors="coerce")
        except:
            continue
    # fallback: let pandas auto-infer
    return pd.to_datetime(date_series, errors="coerce")

# -----------------------------
# Process each CSV
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"\n========== Processing: {file} ==========")

        # Load CSV
        df = pd.read_csv(file_path)

        # Parse dates
        df['Date'] = parse_dates(df['Date'])
        if df['Date'].isna().sum() > 0:
            print(f"⚠️ Warning: {df['Date'].isna().sum()} invalid dates found and dropped")
            df = df.dropna(subset=['Date'])

        df = df.sort_values('Date')

        # Handle missing values
        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'] = df['Average Price'].fillna(mean_value)
            print(f"Filled missing values with mean: {mean_value:.2f}")

        # Prepare data for Prophet
        prophet_df = df[['Date', 'Average Price']].rename(columns={'Date': 'ds', 'Average Price': 'y'})

        # Build and fit Prophet model
        model = Prophet(daily_seasonality=True)
        model.fit(prophet_df)

        # Predict for existing dates
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)
        df['Predicted'] = forecast['yhat'].values

        # Round Actual and Predicted to 2 decimals
        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Save CSV with Actual vs Predicted
        output_df = df[['Date', 'Average Price', 'Predicted']].rename(columns={'Average Price': 'Actual'})
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_prophet_results.csv"))
        output_df.to_csv(updated_csv_path, index=False)

        # Calculate metrics
        y_true = df['Average Price'].values
        y_pred = df['Predicted'].values

        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy = round(100 - mape, 2)

        # Print metrics
        print(f"Metrics for {file}:")
        print(f"  MAE      : {mae}")
        print(f"  RMSE     : {rmse}")
        print(f"  R²       : {r2}")
        print(f"  MAPE     : {mape}%")
        print(f"  Accuracy : {accuracy}%")

        # Save metrics
        metrics_list.append({
            'File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_prophet_model.pkl"))
        joblib.dump(model, model_file)

        # Save Graph
        plt.figure(figsize=(10, 6))
        plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue', linewidth=2)
        plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red', linewidth=2, linestyle='--')
        plt.title(f"Actual vs Predicted - {file}", fontsize=14)
        plt.xlabel("Date", fontsize=12)
        plt.ylabel("Price", fontsize=12)
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()

        graph_path = os.path.join(output_graphs, file.replace(".csv", "_prophet_graph.png"))
        plt.savefig(graph_path)
        plt.close()

# Save all metrics to CSV
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)
print(f"\n✅ All districts processed! Metrics saved to {output_metrics_csv}, CSV results saved, and graphs generated.")


========== Processing: tomato_Bangalore_weekly.csv ==========


07:47:29 - cmdstanpy - INFO - Chain [1] start processing
07:47:30 - cmdstanpy - INFO - Chain [1] done processing


Metrics for tomato_Bangalore_weekly.csv:
  MAE      : 679.08
  RMSE     : 993.13
  R²       : 0.29
  MAPE     : 53.53%
  Accuracy : 46.47%

========== Processing: tomato_Belgaum_weekly.csv ==========


07:47:31 - cmdstanpy - INFO - Chain [1] start processing
07:47:31 - cmdstanpy - INFO - Chain [1] done processing


Metrics for tomato_Belgaum_weekly.csv:
  MAE      : 389.3
  RMSE     : 718.84
  R²       : 0.58
  MAPE     : 22.02%
  Accuracy : 77.98%

========== Processing: tomato_Bellary_weekly.csv ==========


...

Metrics for tomato_Shimoga_weekly.csv:
  MAE      : 536.95
  RMSE     : 829.8
  R²       : 0.26
  MAPE     : 51.65%
  Accuracy : 48.35%

========== Processing: tomato_Tumkur_weekly.csv ==========


07:47:52 - cmdstanpy - INFO - Chain [1] start processing
07:47:52 - cmdstanpy - INFO - Chain [1] done processing


Metrics for tomato_Tumkur_weekly.csv:
  MAE      : 296.83
  RMSE     : 476.66
  R²       : 0.89
  MAPE     : 29.99%
  Accuracy : 70.01%

========== Processing: tomato_Udupi_weekly.csv ==========


07:47:53 - cmdstanpy - INFO - Chain [1] start processing
07:47:54 - cmdstanpy - INFO - Chain [1] done processing


Metrics for tomato_Udupi_weekly.csv:
  MAE      : 693.61
  RMSE     : 1034.8
  R²       : 0.35
  MAPE     : 42.19%
  Accuracy : 57.81%

✅ All districts processed! Metrics saved to prophet_metrics.csv, CSV results saved, and graphs generated.


In [ ]:
#ARIMA

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import warnings
warnings.filterwarnings("ignore")

# -----------------------------
# Create output directories
# -----------------------------
input_folder = "dataset"
output_models = "arima_output_models"
output_csv = "arima_output_csv"
output_graphs = "arima_output_graphs"
output_logs = "arima_output_logs"
output_metrics = "arima_output_metrics"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)
os.makedirs(output_metrics, exist_ok=True)

# Metrics collector
all_metrics = []

# -----------------------------
# Helper function for flexible date parsing
# -----------------------------
def parse_dates(date_series):
    """Try multiple date formats until successful."""
    for fmt in ("%Y-%m-%d", "%d-%m-%Y", "%d/%m/%Y", "%Y/%m/%d"):
        try:
            return pd.to_datetime(date_series, format=fmt)
        except Exception:
            continue
    return pd.to_datetime(date_series, errors="coerce")  # fallback

# -----------------------------
# Process each CSV file
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"Processing: {file}")

        try:
            # Load CSV
            df = pd.read_csv(file_path)

            # Handle multiple date formats
            df['Date'] = parse_dates(df['Date'])
            df = df.dropna(subset=['Date'])
            df = df.sort_values('Date')
            df.set_index('Date', inplace=True)

            # Handle NaN values in Average Price
            df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

            # Add moving averages
            df['MA_7'] = df['Average Price'].rolling(window=7).mean()
            df['MA_30'] = df['Average Price'].rolling(window=30).mean()

            # ARIMA parameters (static, can be tuned later)
            p, d, q = 5, 1, 0
            model = ARIMA(df['Average Price'], order=(p, d, q))
            model_fit = model.fit()

            # Save ARIMA model
            model_file = os.path.join(output_models, file.replace(".csv", "_arima_model.pkl"))
            joblib.dump(model_fit, model_file)

            # Predictions
            predictions = model_fit.predict(start=0, end=len(df)-1, typ='levels')
            df['Predicted'] = predictions

            # Round Actual and Predicted to 2 decimals
            df['Average Price'] = df['Average Price'].round(2)
            df['Predicted'] = df['Predicted'].round(2)

            # Save Actual vs Predicted CSV
            comparison_df = pd.DataFrame({
                "Date": df.index,
                "Actual": df['Average Price'],
                "Predicted": df['Predicted']
            })
            comparison_csv = os.path.join(output_csv, file.replace(".csv", "_arima_actual_vs_predicted.csv"))
            comparison_df.to_csv(comparison_csv, index=False)

            # Calculate metrics
            mae = round(mean_absolute_error(df['Average Price'], df['Predicted']), 2)
            rmse = round(np.sqrt(mean_squared_error(df['Average Price'], df['Predicted'])), 2)
            r2 = round(r2_score(df['Average Price'], df['Predicted']), 2)
            mape = round(np.mean(np.abs((df['Average Price'] - df['Predicted']) / df['Average Price'])) * 100, 2)
            accuracy = round(100 - mape, 2)

            all_metrics.append({
                "District": file.replace(".csv", ""),
                "MAE": mae,
                "RMSE": rmse,
                "R2": r2,
                "MAPE(%)": mape,
                "Accuracy (%)": accuracy
            })

            # Print metrics
            print(f"Metrics for {file}:")
            print(f"  MAE      : {mae}")
            print(f"  RMSE     : {rmse}")
            print(f"  R²       : {r2}")
            print(f"  MAPE     : {mape}%")
            print(f"  Accuracy : {accuracy}%\n")

            # Save graph
            plt.figure(figsize=(12,6))
            plt.plot(df.index, df['Average Price'], label="Actual", color="blue", linewidth=2)
            plt.plot(df.index, df['Predicted'], label="Predicted (ARIMA)", color="red", linestyle="dashed", linewidth=2)
            plt.plot(df.index, df['MA_7'], label="MA 7", color="orange")
            plt.plot(df.index, df['MA_30'], label="MA 30", color="green")
            plt.xlabel("Date")
            plt.ylabel("Average Price")
            plt.title(f"Price Prediction (ARIMA) - {file}")
            plt.legend()
            plt.grid(True)
            graph_file = os.path.join(output_graphs, file.replace(".csv", "_arima_graph.png"))
            plt.savefig(graph_file)
            plt.close()

            # Save logs
            log_file = os.path.join(output_logs, file.replace(".csv", "_arima_training.txt"))
            with open(log_file, "w", encoding="utf-8") as f:
                f.write(f"ARIMA model fitted for {file}.\n")
                f.write(f"ARIMA Order: (p={p}, d={d}, q={q})\n")
                f.write(f"MAE: {mae}, RMSE: {rmse}, R²: {r2}, MAPE: {mape}%, Accuracy: {accuracy}%\n")
                f.write("Predictions, Graph, Logs saved.\n")

            print(f"Done with {file} | Model, CSV, Graph, Metrics, Log saved.\n")

        except Exception as e:
            print(f"Error processing {file}: {e}")

# -----------------------------
# Save all metrics in one CSV
# -----------------------------
if all_metrics:
    metrics_df = pd.DataFrame(all_metrics)
    metrics_csv = os.path.join(output_metrics, "arima_all_districts_metrics.csv")
    metrics_df.to_csv(metrics_csv, index=False)
    print(f"All metrics saved to {metrics_csv}")

Processing: tomato_Bangalore_weekly.csv
Metrics for tomato_Bangalore_weekly.csv:
  MAE      : 213.78
  RMSE     : 378.68
  R²       : 0.9
  MAPE     : 13.09%
  Accuracy : 86.91%

Done with tomato_Bangalore_weekly.csv | Model, CSV, Graph, Metrics, Log saved.

Processing: tomato_Belgaum_weekly.csv
Metrics for tomato_Belgaum_weekly.csv:
  MAE      : 104.99
  RMSE     : 303.92
  R²       : 0.93
  MAPE     : 4.25%
  Accuracy : 95.75%

Done with tomato_Belgaum_weekly.csv | Model, CSV, Graph, Metrics, Log saved.

Processing: tomato_Bellary_weekly.csv
Metrics for tomato_Bellary_weekly.csv:
  MAE      : 103.58
  RMSE     : 213.71
  R²       : 0.88
  MAPE     : 12.02%
  Accuracy : 87.98%

Done with tomato_Bellary_weekly.csv | Model, CSV, Graph, Metrics, Log saved.

Processing: tomato_Chamrajnagar_weekly.csv
Metrics for tomato_Chamrajnagar_weekly.csv:
  MAE      : 112.61
  RMSE     : 263.25
  R²       : 0.94
  MAPE     : 6.81%
  Accuracy : 93.19%

Done with tomato_Chamrajnagar_weekly.csv | Model,

In [ ]:
#SARIMAX

In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import joblib

warnings.filterwarnings("ignore")  # Ignore warnings

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "sarimax_output_models"
output_csv = "sarimax_output_csv"
output_graphs = "sarimax_output_graphs"
output_metrics_csv = "sarimax_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"========== Processing: {file} ==========")

        # Load CSV
        df = pd.read_csv(file_path)

        # Handle multiple date formats
        try:
            df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        except Exception as e:
            print(f"Date conversion failed: {e}")
            continue

        df = df.dropna(subset=['Date'])  # Drop rows where date conversion failed
        df = df.sort_values('Date')
        df.set_index('Date', inplace=True)

        # Handle missing values
        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'].fillna(mean_value, inplace=True)
            print(f"Filled missing values with mean: {mean_value:.2f}")

        # Add moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()

        # SARIMAX order (tune if needed)
        order = (1, 1, 1)
        seasonal_order = (1, 1, 1, 7)

        # Fit SARIMAX model
        model = SARIMAX(df['Average Price'],
                        order=order,
                        seasonal_order=seasonal_order,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        model_fit = model.fit(disp=False)

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_sarimax_model.pkl"))
        joblib.dump(model_fit, model_file)

        # Predictions
        df['Predicted'] = model_fit.predict(start=0, end=len(df)-1)

        # Round Actual and Predicted to 2 decimals
        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Calculate metrics
        y_true = df['Average Price'].values
        y_pred = df['Predicted'].values
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy = round(100 - mape, 2)

        # Display metrics
        print(f"Metrics for {file}:")
        print(f"  MAE      : {mae}")
        print(f"  RMSE     : {rmse}")
        print(f"  R²       : {r2}")
        print(f"  MAPE     : {mape}%")
        print(f"  Accuracy : {accuracy}%\n")

        # Save metrics to list
        metrics_list.append({
            'File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # Save Actual vs Predicted CSV
        actual_vs_pred = df[['Average Price', 'Predicted']].reset_index()
        actual_vs_pred.rename(columns={'Average Price': 'Actual'}, inplace=True)
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_sarimax_actual_vs_predicted.csv"))
        actual_vs_pred.to_csv(updated_csv_path, index=False)

        # Save graph
        plt.figure(figsize=(12,6))
        plt.plot(df.index, df['Average Price'], label="Actual", color="blue")
        plt.plot(df.index, df['MA_7'], label="MA 7", color="orange")
        plt.plot(df.index, df['MA_30'], label="MA 30", color="green")
        plt.plot(df.index, df['Predicted'], label="Predicted (SARIMAX)", color="red", linestyle="dashed")
        plt.xlabel("Date")
        plt.ylabel("Average Price")
        plt.title(f"Price Prediction (SARIMAX) - {file}")
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_sarimax_graph.png"))
        plt.savefig(graph_file)
        plt.close()

# -----------------------------
# Save all metrics to CSV
# -----------------------------
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)

print(f"✅ All districts processed! Metrics saved to {output_metrics_csv}, predictions CSVs, graphs, and models generated.")

========== Processing: tomato_Bangalore_weekly.csv ==========
Metrics for tomato_Bangalore_weekly.csv:
  MAE      : 220.91
  RMSE     : 381.99
  R²       : 0.89
  MAPE     : 14.28%
  Accuracy : 85.72%

========== Processing: tomato_Belgaum_weekly.csv ==========
Metrics for tomato_Belgaum_weekly.csv:
  MAE      : 106.1
  RMSE     : 317.93
  R²       : 0.92
  MAPE     : 4.43%
  Accuracy : 95.57%

========== Processing: tomato_Bellary_weekly.csv ==========
Metrics for tomato_Bellary_weekly.csv:
  MAE      : 108.96
  RMSE     : 221.32
  R²       : 0.87
  MAPE     : 12.97%
  Accuracy : 87.03%

========== Processing: tomato_Chamrajnagar_weekly.csv ==========
Metrics for tomato_Chamrajnagar_weekly.csv:
  MAE      : 120.79
  RMSE     : 266.93
  R²       : 0.94
  MAPE     : 8.32%
  Accuracy : 91.68%

========== Processing: tomato_Chikmagalur_weekly.csv ==========
Metrics for tomato_Chikmagalur_weekly.csv:
  MAE      : 103.56
  RMSE     : 182.22
  R²       : 0.93
  MAPE     : 9.82%
  Accuracy : 

In [ ]:
#TAT + MHA

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_output_models"
output_csv = "tat_output_csv"
output_graphs = "tat_output_graphs"
output_logs = "tat_output_logs"
metrics_file = "tat_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Define Temporal Attention Transformer Model
# -----------------------------
def build_tat_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)  # (look_back, 1)
    
    # Multi-Head Attention
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(inputs, inputs)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual connection
    
    # Feed-Forward Network
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual connection
    
    # Global Pooling
    x = layers.GlobalAveragePooling1D()(x)
    
    # Output
    outputs = layers.Dense(1)(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"🚀 Processing: {file}")

        # Load CSV
        df = pd.read_csv(file_path)
        
        # Handle multiple date formats
        try:
            df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        except Exception as e:
            print(f"Date conversion failed: {e}")
            continue
        
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Replace NaN in 'Average Price' with column mean
        df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)

        # Add moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()

        # Replace NaN in moving averages with column mean
        df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
        df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

        # Prepare data using MinMaxScaler
        values = df[['Average Price']].values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)

        # Create sequences
        X, y = create_dataset(scaled_values, look_back)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build model
        model = build_tat_model(input_shape=(look_back,1))
        model.summary()

        # Train model
        history = model.fit(X, y, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

        # Save training logs
        log_file = os.path.join(output_logs, file.replace(".csv", "_tat_training.txt"))
        with open(log_file, "w") as f:
            f.write("Training Loss per Epoch:\n")
            for i, loss in enumerate(history.history['loss']):
                f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

        # Predictions
        predictions = model.predict(X)
        predictions_rescaled = scaler.inverse_transform(predictions)
        df['Predicted'] = [np.nan]*look_back + list(predictions_rescaled.flatten())

        # Round Actual and Predicted to 2 decimals
        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Compute metrics
        y_true = df['Average Price'].values[look_back:]
        y_pred = predictions_rescaled.flatten()
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy = round(100 - mape, 2)

        metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_tat_model.h5"))
        model.save(model_file)

        # Save CSV
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_updated.csv"))
        df.to_csv(updated_csv_path, index=False)

        # Save graph
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Average Price'], label='Ground Truth', color='blue')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT)', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f'Price Prediction (TAT) - {file}')
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_graph.png"))
        plt.savefig(graph_file)
        plt.close()

        print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%\n")

# Save all metrics
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: tomato_Bangalore_weekly.csv


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention          │ (None, 30, 1)             │           1,793 │ input_layer[0][0],         │
│ (MultiHeadAttention)          │                           │                 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 30, 1)             │               0 │ multi_head_attention[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 30, 1)             │               0 │ dropout_1[0][0],           │
│                               │                           │                 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization           │ (None, 30, 1)             │               2 │ add[0][0]                  │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 30, 128)           │             256 │ layer_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 30, 1)             │             129 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_1 (Add)                   │ (None, 30, 1)             │               0 │ layer_normalization[0][0], │
│                               │                           │                 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_1         │ (None, 30, 1)             │               2 │ add_1[0][0]                │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d      │ (None, 1)                 │               0 │ layer_normalization_1[0][… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 1)                 │               2 │ global_average_pooling1d[… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,184 (8.53 KB)

 Trainable params: 2,184 (8.53 KB)

...

 Total params: 2,184 (8.53 KB)

 Trainable params: 2,184 (8.53 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.0174 - mae: 0.0879 - val_loss: 0.0779 - val_mae: 0.1900
Epoch 2/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0116 - mae: 0.0696 - val_loss: 0.0653 - val_mae: 0.1621
Epoch 3/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0103 - mae: 0.0712 - val_loss: 0.0611 - val_mae: 0.1539
Epoch 4/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0101 - mae: 0.0730 - val_loss: 0.0597 - val_mae: 0.1517
Epoch 5/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0102 - mae: 0.0740 - val_loss: 0.0601 - val_mae: 0.1523
Epoch 6/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0101 - mae: 0.0736 - val_loss: 0.0597 - val_mae: 0.1518
Epoch 7/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0101 - mae: 0.0736 - val_loss: 0.0601 - val_mae: 0.1523
Epoch 8/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0102 - mae: 0.0733 - val_loss: 0.0603 - val_mae: 0.1526
Epoch 9/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0

✅ Done with tomato_Udupi_weekly.csv | MAE=834.91, RMSE=1324.9, R2=-0.03, MAPE=46.58%, Accuracy=53.42%

📊 Metrics saved to tat_metrics.csv


In [ ]:
#TAT + MQA

In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_mqa_output_models"
output_csv = "tat_mqa_output_csv"
output_graphs = "tat_mqa_output_graphs"
output_logs = "tat_mqa_output_logs"
metrics_file = "tat_mqa_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Multi-Query Attention Layer
# -----------------------------
class MultiQueryAttention(layers.Layer):
    def __init__(self, num_heads, key_dim, dropout_rate=0.1, **kwargs):
        super(MultiQueryAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.q_dense = [layers.Dense(key_dim) for _ in range(num_heads)]
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.dropout = layers.Dropout(dropout_rate)
        self.output_dense = layers.Dense(key_dim)

    def call(self, x):
        K = self.k_dense(x)
        V = self.v_dense(x)
        head_outputs = []

        for q_layer in self.q_dense:
            Q = q_layer(x)
            scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
            attention_weights = tf.nn.softmax(scores, axis=-1)
            attention_output = tf.matmul(attention_weights, V)
            head_outputs.append(attention_output)

        concat = tf.concat(head_outputs, axis=-1)
        output = self.output_dense(concat)
        output = self.dropout(output)
        return output

    def get_config(self):
        config = super(MultiQueryAttention, self).get_config()
        config.update({
            "num_heads": self.num_heads,
            "key_dim": self.key_dim,
            "dropout_rate": self.dropout_rate
        })
        return config

# -----------------------------
# TAT-MQA Model
# -----------------------------
def build_tat_mqa_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    x = MultiQueryAttention(num_heads=num_heads, key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"🚀 Processing: {file}")

        # Load CSV
        df = pd.read_csv(file_path)

        # Handle multiple date formats
        try:
            df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        except Exception as e:
            print(f"Date conversion failed: {e}")
            continue

        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Handle NaNs
        df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)

        # Moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()
        df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
        df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

        # Prepare data
        values = df[['Average Price']].values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)
        X, y = create_dataset(scaled_values, look_back)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build model
        model = build_tat_mqa_model(input_shape=(look_back,1))
        model.summary()

        # Train model
        history = model.fit(X, y, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

        # Save logs
        log_file = os.path.join(output_logs, file.replace(".csv", "_tat_mqa_training.txt"))
        with open(log_file, "w") as f:
            f.write("Training Loss per Epoch:\n")
            for i, loss in enumerate(history.history['loss']):
                f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

        # Predictions
        predictions = model.predict(X)
        predictions_rescaled = scaler.inverse_transform(predictions)
        df['Predicted'] = [np.nan]*look_back + list(predictions_rescaled.flatten())

        # Round Actual and Predicted
        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Metrics
        y_true = df['Average Price'].values[look_back:]
        y_pred = predictions_rescaled.flatten()
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy = round(100 - mape, 2)
        metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_tat_mqa_model.keras"))
        model.save(model_file)

        # Save CSV
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_mqa_updated.csv"))
        df.to_csv(updated_csv_path, index=False)

        # Save graph
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Average Price'], label='Ground Truth', color='blue')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT-MQA)', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f'Price Prediction (TAT-MQA) - {file}')
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_mqa_graph.png"))
        plt.savefig(graph_file)
        plt.close()

        print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%\n")

# Save metrics
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: tomato_Bangalore_weekly.csv



Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_19 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_query_attention         │ (None, 30, 64)            │          17,216 │ input_layer_19[0][0]       │
│ (MultiQueryAttention)         │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_38 (Add)                  │ (None, 30, 64)            │               0 │ multi_query_attention[0][… │
│                               │                           │                 │ input_layer_19[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_38        │ (None, 30, 64)            │             128 │ add_38[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_64 (Dense)              │ (None, 30, 128)           │           8,320 │ layer_normalization_38[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_65 (Dense)              │ (None, 30, 1)             │             129 │ dense_64[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_39 (Add)                  │ (None, 30, 64)            │               0 │ layer_normalization_38[0]… │
│                               │                           │                 │ dense_65[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_39        │ (None, 30, 64)            │             128 │ add_39[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_19   │ (None, 64)                │               0 │ layer_normalization_39[0]… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_66 (Dense)              │ (None, 1)                 │              65 │ global_average_pooling1d_… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 25,986 (101.51 KB)

 Trainable params: 25,986 (101.51 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_37 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_query_attention_18      │ (None, 30, 64)            │          17,216 │ input_layer_37[0][0]       │
│ (MultiQueryAttention)         │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_74 (Add)                  │ (None, 30, 64)            │               0 │ multi_query_attention_18[… │
│                               │                           │                 │ input_layer_37[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_74        │ (None, 30, 64)            │             128 │ add_74[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_244 (Dense)             │ (None, 30, 128)           │           8,320 │ layer_normalization_74[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_245 (Dense)             │ (None, 30, 1)             │             129 │ dense_244[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_75 (Add)                  │ (None, 30, 64)            │               0 │ layer_normalization_74[0]… │
│                               │                           │                 │ dense_245[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_75        │ (None, 30, 64)            │             128 │ add_75[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_37   │ (None, 64)                │               0 │ layer_normalization_75[0]… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_246 (Dense)             │ (None, 1)                 │              65 │ global_average_pooling1d_… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 25,986 (101.51 KB)

 Trainable params: 25,986 (101.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - loss: 0.3315 - mae: 0.4044 - val_loss: 0.1203 - val_mae: 0.2790
Epoch 2/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0226 - mae: 0.1173 - val_loss: 0.0548 - val_mae: 0.1453
Epoch 3/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0148 - mae: 0.0925 - val_loss: 0.0463 - val_mae: 0.1390
Epoch 4/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0137 - mae: 0.0893 - val_loss: 0.0485 - val_mae: 0.1389
Epoch 5/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0143 - mae: 0.0887 - val_loss: 0.0458 - val_mae: 0.1393
Epoch 6/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0146 - mae: 0.0933 - val_loss: 0.0500 - val_mae: 0.1400
Epoch 7/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0135 - mae: 0.0878 - val_loss: 0.0439 - val_mae: 0.1418
Epoch 8/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0129 - mae: 0.0853 - val_loss: 0.0452 - val_mae: 0.1398
Epoch 9/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - l

In [ ]:
#TAT+GQA

In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_gqa_output_models"
output_csv = "tat_gqa_output_csv"
output_graphs = "tat_gqa_output_graphs"
output_logs = "tat_gqa_output_logs"
metrics_file = "tat_gqa_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Grouped Query Attention Layer
# -----------------------------
class GroupedQueryAttention(layers.Layer):
    def __init__(self, num_groups=2, key_dim=64, dropout_rate=0.1, **kwargs):
        super(GroupedQueryAttention, self).__init__(**kwargs)
        self.num_groups = num_groups
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.q_dense_groups = [layers.Dense(key_dim) for _ in range(num_groups)]
        self.dropout = layers.Dropout(dropout_rate)
        self.output_dense = layers.Dense(key_dim)

    def call(self, x):
        K = self.k_dense(x)
        V = self.v_dense(x)
        group_outputs = []
        for q_layer in self.q_dense_groups:
            Q = q_layer(x)
            scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
            attention_weights = tf.nn.softmax(scores, axis=-1)
            attn_output = tf.matmul(attention_weights, V)
            group_outputs.append(attn_output)
        concat = tf.concat(group_outputs, axis=-1)
        output = self.output_dense(concat)
        output = self.dropout(output)
        return output

    def get_config(self):
        config = super(GroupedQueryAttention, self).get_config()
        config.update({
            "num_groups": self.num_groups,
            "key_dim": self.key_dim,
            "dropout_rate": self.dropout_rate
        })
        return config

# -----------------------------
# TAT-GQA Model
# -----------------------------
def build_tat_gqa_model(input_shape, d_model=64, num_groups=2, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    x = GroupedQueryAttention(num_groups=num_groups, key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"🚀 Processing: {file}")

        # Load CSV
        df = pd.read_csv(file_path)

        # Handle multiple date formats
        try:
            df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        except Exception as e:
            print(f"Date conversion failed: {e}")
            continue

        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Handle NaNs
        df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)

        # Moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()
        df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
        df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

        # Prepare data
        values = df[['Average Price']].values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)
        X, y = create_dataset(scaled_values, look_back)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build model
        model = build_tat_gqa_model(input_shape=(look_back,1), num_groups=2)
        model.summary()

        # Train model
        history = model.fit(X, y, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

        # Save logs
        log_file = os.path.join(output_logs, file.replace(".csv", "_tat_gqa_training.txt"))
        with open(log_file, "w") as f:
            f.write("Training Loss per Epoch:\n")
            for i, loss in enumerate(history.history['loss']):
                f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

        # Predictions
        predictions = model.predict(X)
        predictions_rescaled = scaler.inverse_transform(predictions)
        df['Predicted'] = [np.nan]*look_back + list(predictions_rescaled.flatten())

        # Round Actual and Predicted
        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Metrics
        y_true = df['Average Price'].values[look_back:]
        y_pred = predictions_rescaled.flatten()
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy = round(100 - mape, 2)
        metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_tat_gqa_model.keras"))
        model.save(model_file)

        # Save CSV
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_gqa_updated.csv"))
        df.to_csv(updated_csv_path, index=False)

        # Save graph
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Average Price'], label='Ground Truth', color='blue')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT-GQA)', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f'Price Prediction (TAT-GQA) - {file}')
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_gqa_graph.png"))
        plt.savefig(graph_file)
        plt.close()

        print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%\n")

# Save metrics
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: tomato_Bangalore_weekly.csv


Model: "functional_38"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_38 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ grouped_query_attention       │ (None, 30, 64)            │           8,768 │ input_layer_38[0][0]       │
│ (GroupedQueryAttention)       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_76 (Add)                  │ (None, 30, 64)            │               0 │ grouped_query_attention[0… │
│                               │                           │                 │ input_layer_38[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_76        │ (None, 30, 64)            │             128 │ add_76[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_252 (Dense)             │ (None, 30, 128)           │           8,320 │ layer_normalization_76[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_253 (Dense)             │ (None, 30, 1)             │             129 │ dense_252[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_77 (Add)                  │ (None, 30, 64)            │               0 │ layer_normalization_76[0]… │
│                               │                           │                 │ dense_253[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_77        │ (None, 30, 64)            │             128 │ add_77[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_38   │ (None, 64)                │               0 │ layer_normalization_77[0]… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_254 (Dense)             │ (None, 1)                 │              65 │ global_average_pooling1d_… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 17,538 (68.51 KB)

 Trainable params: 17,538 (68.51 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_56 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ grouped_query_attention_18    │ (None, 30, 64)            │           8,768 │ input_layer_56[0][0]       │
│ (GroupedQueryAttention)       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_112 (Add)                 │ (None, 30, 64)            │               0 │ grouped_query_attention_1… │
│                               │                           │                 │ input_layer_56[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_112       │ (None, 30, 64)            │             128 │ add_112[0][0]              │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_396 (Dense)             │ (None, 30, 128)           │           8,320 │ layer_normalization_112[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_397 (Dense)             │ (None, 30, 1)             │             129 │ dense_396[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_113 (Add)                 │ (None, 30, 64)            │               0 │ layer_normalization_112[0… │
│                               │                           │                 │ dense_397[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_113       │ (None, 30, 64)            │             128 │ add_113[0][0]              │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_56   │ (None, 64)                │               0 │ layer_normalization_113[0… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_398 (Dense)             │ (None, 1)                 │              65 │ global_average_pooling1d_… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 17,538 (68.51 KB)

 Trainable params: 17,538 (68.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - loss: 0.1569 - mae: 0.2848 - val_loss: 0.0435 - val_mae: 0.1639
Epoch 2/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0185 - mae: 0.1061 - val_loss: 0.0494 - val_mae: 0.1395
Epoch 3/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0146 - mae: 0.0936 - val_loss: 0.0458 - val_mae: 0.1392
Epoch 4/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0148 - mae: 0.0911 - val_loss: 0.0477 - val_mae: 0.1387
Epoch 5/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0148 - mae: 0.0941 - val_loss: 0.0472 - val_mae: 0.1388
Epoch 6/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0142 - mae: 0.0890 - val_loss: 0.0458 - val_mae: 0.1392
Epoch 7/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0138 - mae: 0.0865 - val_loss: 0.0427 - val_mae: 0.1469
Epoch 8/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0142 - mae: 0.0888 - val_loss: 0.0441 - val_mae: 0.1410
Epoch 9/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - lo

In [ ]:
#TAT+HA

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_ha_output_models"
output_csv = "tat_ha_output_csv"
output_graphs = "tat_ha_output_graphs"
output_logs = "tat_ha_output_logs"
metrics_file = "tat_ha_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Local Attention Layer
# -----------------------------
class LocalAttention(layers.Layer):
    def __init__(self, key_dim=64, dropout_rate=0.1, **kwargs):
        super(LocalAttention, self).__init__(**kwargs)
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.q_dense = layers.Dense(key_dim)
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.dropout = layers.Dropout(dropout_rate)

    def call(self, x):
        Q = self.q_dense(x)
        K = self.k_dense(x)
        V = self.v_dense(x)
        scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(tf.shape(K)[-1], tf.float32))
        weights = tf.nn.softmax(scores, axis=-1)
        output = tf.matmul(weights, V)
        output = self.dropout(output)
        return output

# -----------------------------
# Hierarchical Attention Model
# -----------------------------
def build_tat_ha_model(input_shape, d_model=64, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    local_attn = LocalAttention(key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    local_attn = layers.LayerNormalization(epsilon=1e-6)(local_attn + inputs)
    global_attn = layers.MultiHeadAttention(num_heads=4, key_dim=d_model)(local_attn, local_attn)
    global_attn = layers.Dropout(dropout_rate)(global_attn)
    global_attn = layers.LayerNormalization(epsilon=1e-6)(global_attn + local_attn)
    ff = layers.Dense(ff_dim, activation='relu')(global_attn)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(ff + global_attn)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"🚀 Processing: {file}")

        # Load CSV
        df = pd.read_csv(file_path)

        # Handle multiple date formats
        df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Handle missing Average Price
        df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)

        # Moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()
        df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
        df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

        # Prepare data
        values = df[['Average Price']].values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)
        X, y = create_dataset(scaled_values, look_back)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build model
        model = build_tat_ha_model(input_shape=(look_back,1))
        model.summary()

        # Train model
        history = model.fit(X, y, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

        # Save logs
        log_file = os.path.join(output_logs, file.replace(".csv", "_tat_ha_training.txt"))
        with open(log_file, "w") as f:
            f.write("Training Loss per Epoch:\n")
            for i, loss in enumerate(history.history['loss']):
                f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

        # Predictions
        predictions = model.predict(X)
        predictions_rescaled = scaler.inverse_transform(predictions)
        df['Predicted'] = [np.nan]*look_back + list(predictions_rescaled.flatten())

        # Round Actual & Predicted
        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Metrics
        y_true = df['Average Price'].values[look_back:]
        y_pred = predictions_rescaled.flatten()
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred)/y_true))*100, 2)
        accuracy = round(100 - mape, 2)
        metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

        # Save model as .pkl
        model_file = os.path.join(output_models, file.replace(".csv", "_tat_ha_model.pkl"))
        joblib.dump(model, model_file)

        # Save CSV
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_ha_updated.csv"))
        df.to_csv(updated_csv_path, index=False)

        # Save graph
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Average Price'], label='Ground Truth', color='blue')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT-HA)', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f'Price Prediction (TAT-HA) - {file}')
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_ha_graph.png"))
        plt.savefig(graph_file)
        plt.close()

        print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%\n")

# Save metrics
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: tomato_Bangalore_weekly.csv


Model: "functional_57"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_57 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ local_attention               │ (None, 30, 64)            │             384 │ input_layer_57[0][0]       │
│ (LocalAttention)              │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_114 (Add)                 │ (None, 30, 64)            │               0 │ local_attention[0][0],     │
│                               │                           │                 │ input_layer_57[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_114       │ (None, 30, 64)            │             128 │ add_114[0][0]              │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention_19       │ (None, 30, 64)            │          66,368 │ layer_normalization_114[0… │
│ (MultiHeadAttention)          │                           │                 │ layer_normalization_114[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_78 (Dropout)          │ (None, 30, 64)            │               0 │ multi_head_attention_19[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_115 (Add)                 │ (None, 30, 64)            │               0 │ dropout_78[0][0],          │
│                               │                           │                 │ layer_normalization_114[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_115       │ (None, 30, 64)            │             128 │ add_115[0][0]              │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_402 (Dense)             │ (None, 30, 128)           │           8,320 │ layer_normalization_115[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_403 (Dense)             │ (None, 30, 1)             │             129 │ dense_402[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_116 (Add)                 │ (None, 30, 64)            │               0 │ dense_403[0][0],           │
│                               │                           │                 │ layer_normalization_115[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_116       │ (None, 30, 64)            │             128 │ add_116[0][0]              │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_57   │ (None, 64)                │               0 │ layer_normalization_116[0… │
│ (GlobalAveragePooling1D)      │                           │               

 Total params: 75,650 (295.51 KB)

 Trainable params: 75,650 (295.51 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_75 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ local_attention_18            │ (None, 30, 64)            │             384 │ input_layer_75[0][0]       │
│ (LocalAttention)              │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_168 (Add)                 │ (None, 30, 64)            │               0 │ local_attention_18[0][0],  │
│                               │                           │                 │ input_layer_75[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_168       │ (None, 30, 64)            │             128 │ add_168[0][0]              │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention_37       │ (None, 30, 64)            │          66,368 │ layer_normalization_168[0… │
│ (MultiHeadAttention)          │                           │                 │ layer_normalization_168[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_132 (Dropout)         │ (None, 30, 64)            │               0 │ multi_head_attention_37[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_169 (Add)                 │ (None, 30, 64)            │               0 │ dropout_132[0][0],         │
│                               │                           │                 │ layer_normalization_168[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_169       │ (None, 30, 64)            │             128 │ add_169[0][0]              │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_510 (Dense)             │ (None, 30, 128)           │           8,320 │ layer_normalization_169[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_511 (Dense)             │ (None, 30, 1)             │             129 │ dense_510[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_170 (Add)                 │ (None, 30, 64)            │               0 │ dense_511[0][0],           │
│                               │                           │                 │ layer_normalization_169[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_170       │ (None, 30, 64)            │             128 │ add_170[0][0]              │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_75   │ (None, 64)                │               0 │ layer_normalization_170[0… │
│ (GlobalAveragePooling1D)      │                           │               

 Total params: 75,650 (295.51 KB)

 Trainable params: 75,650 (295.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 13s 39ms/step - loss: 0.2062 - mae: 0.3160 - val_loss: 0.0670 - val_mae: 0.1664
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0179 - mae: 0.0998 - val_loss: 0.0585 - val_mae: 0.1508
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0157 - mae: 0.0957 - val_loss: 0.0509 - val_mae: 0.1410
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0152 - mae: 0.0936 - val_loss: 0.0499 - val_mae: 0.1400
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0132 - mae: 0.0852 - val_loss: 0.0449 - val_mae: 0.1401
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0143 - mae: 0.0910 - val_loss: 0.0477 - val_mae: 0.1388
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0142 - mae: 0.0898 - val_loss: 0.0441 - val_mae: 0.1410
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0128 - mae: 0.0848 - val_loss: 0.0478 - val_mae: 0.1387
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.01

In [9]:
#LSTM

In [12]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Paths
input_folder = "dataset"
output_models = "output_lstm_models"
output_csv = "output_lstm_csv"
output_graphs = "output_lstm_graphs"
output_logs = "output_lstm_logs"
metrics_file = os.path.join(output_csv, "lstm_metrics.csv")

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# Function to create sequences
def create_sequences(data, seq_length=5):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Metrics storage
metrics_list = []

# Process each CSV file
seq_length = 5
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        district_name = file.split(".")[0]
        print(f"\n========== Processing: {file} ==========")
        
        df = pd.read_csv(os.path.join(input_folder, file))
        df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

        df['MA_7'] = df['Average Price'].rolling(window=7).mean().fillna(df['Average Price'].mean())
        df['MA_30'] = df['Average Price'].rolling(window=30).mean().fillna(df['Average Price'].mean())

        scaler = MinMaxScaler()
        prices_scaled = scaler.fit_transform(df['Average Price'].values.reshape(-1, 1))

        X, y = create_sequences(prices_scaled, seq_length)

        split = int(0.8 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(seq_length, 1)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])

        history = model.fit(
            X_train, y_train, 
            epochs=20, batch_size=32,
            validation_data=(X_test, y_test),
            verbose=1
        )

        y_pred_scaled = model.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred_scaled)
        y_true = scaler.inverse_transform(y_test.reshape(-1,1))

        y_true_round = np.round(y_true, 2)
        y_pred_round = np.round(y_pred, 2)

        df_pred = df.iloc[seq_length + split:].copy()
        df_pred['Predicted'] = y_pred_round.flatten()
        df_pred['Average Price'] = y_true_round.flatten()
        df_pred.to_csv(os.path.join(output_csv, f"{district_name}_lstm_updated.csv"), index=False)

        mae_val = round(mean_absolute_error(y_true, y_pred), 2)
        rmse_val = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)  # FIXED
        r2_val = round(r2_score(y_true, y_pred), 2)
        mape_val = round(np.mean(np.abs((y_true - y_pred)/y_true))*100, 2)
        accuracy_val = round(100 - mape_val, 2)

        metrics_list.append([district_name, mae_val, rmse_val, r2_val, mape_val, accuracy_val])

        print(f"✅ Done with {file} | MAE={mae_val}, RMSE={rmse_val}, R2={r2_val}, MAPE={mape_val}%, Accuracy={accuracy_val}%")

        joblib.dump(model, os.path.join(output_models, f"{district_name}_lstm_model.pkl"))

        plt.figure(figsize=(12,6))
        plt.plot(df_pred['Date'], df_pred['Average Price'], label='Actual', color='blue')
        plt.plot(df_pred['Date'], df_pred['Predicted'], label='Predicted', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f"LSTM Predictions for {district_name}")
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_graphs, f"{district_name}_lstm_graph.png"))
        plt.close()

        plt.figure(figsize=(8,4))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.title(f"Training Loss for {district_name}")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(os.path.join(output_graphs, f"{district_name}_lstm_training_loss.png"))
        plt.close()

metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)

print(f"\n📊 Metrics saved to {metrics_file}")
print("\nAll districts processed successfully!")


========== Processing: tomato_Bangalore_weekly.csv ==========
Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - loss: 0.0240 - mae: 0.1067 - val_loss: 0.0601 - val_mae: 0.1643
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0123 - mae: 0.0696 - val_loss: 0.0354 - val_mae: 0.1156
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0085 - mae: 0.0666 - val_loss: 0.0282 - val_mae: 0.1053
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0066 - mae: 0.0571 - val_loss: 0.0233 - val_mae: 0.0991
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0053 - mae: 0.0498 - val_loss: 0.0210 - val_mae: 0.0975
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0049 - mae: 0.0479 - val_loss: 0.0205 - val_mae: 0.0965
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0047 - mae: 0.0469 - val_loss: 0.0186 - val_mae: 0.0912
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0045 - mae: 0.0455 - val_loss: 0.0175 - val_mae: 0.0887
E

In [ ]:
#GRU

In [13]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

# Paths
input_folder = "dataset"
output_models = "gru_output_models"
output_csv = "gru_output_csv"
output_graphs = "gru_output_graphs"
output_logs = "gru_output_logs"
metrics_file = os.path.join(output_csv, "gru_metrics.csv")

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# Function to create dataset sequences
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# Store metrics for all files
metrics_list = []

look_back = 30

# Process each CSV file
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        district_name = file.split(".")[0]
        print(f"\n========== Processing: {file} ==========")
        
        df = pd.read_csv(os.path.join(input_folder, file))
        df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

        df['MA_7'] = df['Average Price'].rolling(window=7).mean().fillna(df['Average Price'].mean())
        df['MA_30'] = df['Average Price'].rolling(window=30).mean().fillna(df['Average Price'].mean())

        values = df[['Average Price']].values
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)

        X, y = create_dataset(scaled_values, look_back)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        split = int(0.8 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        # GRU Model
        model = Sequential([
            GRU(64, return_sequences=True, input_shape=(look_back, 1)),
            Dropout(0.2),
            GRU(32),
            Dropout(0.2),
            Dense(1)
        ])
        model.compile(optimizer="adam", loss="mse", metrics=['mae'])

        history = model.fit(
            X_train, y_train,
            epochs=20,
            batch_size=16,
            validation_data=(X_test, y_test),
            verbose=1
        )

        # Save training log
        log_file = os.path.join(output_logs, f"{district_name}_gru_training.txt")
        with open(log_file, "w") as f:
            f.write("Epoch\tTrain_Loss\tVal_Loss\n")
            for i, (loss, val_loss) in enumerate(zip(history.history['loss'], history.history['val_loss'])):
                f.write(f"{i+1}\t{loss:.6f}\t{val_loss:.6f}\n")

        # Predictions
        y_pred_scaled = model.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred_scaled)
        y_true = scaler.inverse_transform(y_test.reshape(-1, 1))

        y_true_round = np.round(y_true, 2)
        y_pred_round = np.round(y_pred, 2)

        # Save prediction CSV
        df_pred = df.iloc[-len(y_true):].copy()
        df_pred['Predicted'] = y_pred_round.flatten()
        df_pred['Average Price'] = y_true_round.flatten()
        df_pred.to_csv(os.path.join(output_csv, f"{district_name}_gru_updated.csv"), index=False)

        # -----------------------------
        # FIXED METRICS (NO ERROR!)
        # -----------------------------
        mae_val = round(mean_absolute_error(y_true, y_pred), 2)
        rmse_val = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)  # FIXED
        r2_val = round(r2_score(y_true, y_pred), 2)
        mape_val = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy_val = round(100 - mape_val, 2)

        metrics_list.append([district_name, mae_val, rmse_val, r2_val, mape_val, accuracy_val])

        print(f"✅ Done with {file} | MAE={mae_val}, RMSE={rmse_val}, R2={r2_val}, MAPE={mape_val}%, Accuracy={accuracy_val}%")

        # Save GRU model
        joblib.dump(model, os.path.join(output_models, f"{district_name}_gru_model.pkl"))

        # Prediction graph
        plt.figure(figsize=(12, 6))
        plt.plot(df_pred['Date'], df_pred['Average Price'], label="Actual", color="blue")
        plt.plot(df_pred['Date'], df_pred['MA_7'], label="MA_7", color="orange")
        plt.plot(df_pred['Date'], df_pred['MA_30'], label="MA_30", color="green")
        plt.plot(df_pred['Date'], df_pred['Predicted'], label="Predicted (GRU)", color="red", linestyle="dashed")
        plt.xlabel("Date")
        plt.ylabel("Average Price")
        plt.title(f"GRU Predictions - {district_name}")
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_graphs, f"{district_name}_gru_graph.png"))
        plt.close()

        # Loss graph
        plt.figure(figsize=(8, 4))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.title(f"GRU Training Loss - {district_name}")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(os.path.join(output_graphs, f"{district_name}_gru_loss.png"))
        plt.close()

# Save all metrics
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)

print("\n📊 Metrics saved to", metrics_file)
print("\n✅ All districts processed successfully!")


========== Processing: tomato_Bangalore_weekly.csv ==========
Epoch 1/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 19s 98ms/step - loss: 0.0088 - mae: 0.0654 - val_loss: 0.0185 - val_mae: 0.0921
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0047 - mae: 0.0465 - val_loss: 0.0118 - val_mae: 0.0717
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0038 - mae: 0.0388 - val_loss: 0.0075 - val_mae: 0.0569
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0031 - mae: 0.0378 - val_loss: 0.0074 - val_mae: 0.0582
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0028 - mae: 0.0345 - val_loss: 0.0078 - val_mae: 0.0604
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.0026 - mae: 0.0327 - val_loss: 0.0059 - val_mae: 0.0519
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 0.0022 - mae: 0.0286 - val_loss: 0.0059 - val_mae: 0.0525
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0021 - mae: 0.0290 - val_loss: 0.0056 - val_mae: 0.0497
